In [18]:
import pandas as pd
import numpy as np
from gurobipy import *

In [37]:
# cargar la cinética de recuperacion
data_name = 'marvin'
cinetica_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/Daniel_inputs/'
block_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/newman1.blocks'
cinetica_file = 'REC_MODEL_T240.csv'
cinetica_df = pd.read_csv(cinetica_path + cinetica_file)
blocks_df = pd.read_csv(block_path, header=None)
rec = cinetica_df['rec90'] # panda series para la recuperacion en t=90

In [38]:
# histograma de la recuperacion en el instante 90
hist = cinetica_df.hist(column='rec90', bins=5)
# calcular nuevos valores de la funcion objetivo
# usando la recuperación al dia 90
nblocks = blocks_df.shape[0]
# algunos datos dados
c_conv = 2204.62
rec_AU = 0.88
price_AU = 12
seilling_AU = 0.2
rec_CU = 0.75
price_CU = 20
seilling_CU = 7.2
proc_cost = 4
mine_cost = 0.9 # por tonn
bv_list_cinetic = list()
bv_list_calc = list()
economic_value_list = list()
economic_value_rec_list = list()
with open('marvin_cinetica.upit', 'w') as output, open(block_path, 'r') as bm:
    output.write('NAME: marvin\nTYPE: UPIT\nNBLOCKS: 53271\nOBJECTIVE_FUNCTION: \n')
    for i in range(nblocks):
        #recuperar datos de bm para calcular valor de cada bloque
        linea_bm = bm.readline()
        linea_lista = linea_bm.split()
        tonn = float(linea_lista[4])
        AU = float(linea_lista[5])
        CU = float(linea_lista[6])
        economic_value = (AU*rec_AU*(price_AU-seilling_AU) + CU*rec[i]/100*(price_CU-seilling_CU)-(mine_cost+proc_cost))*tonn
        economic_value_rec = (AU*rec_AU*(price_AU-seilling_AU) + CU*rec_CU*(price_CU-seilling_CU)-(mine_cost+proc_cost))*tonn
        economic_value_list.append(economic_value)
        economic_value_rec_list.append(economic_value_rec)

ValueError: could not convert string to float: 'FRWS'

In [39]:
upit_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/newman1.upit'
bv_list = list()
with open(upit_path, 'r') as f:
    for i in range(4):
        f.readline()
    for line in f:
        if not line == 'EOF\n':
            lista = line.split()
            bv_list.append(float(lista[1]))

In [21]:
# beneficio con recuperacion de Daniel y calculo posterior (economic value)
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(quicksum(economic_value_list[i]*x[i] for i in range(nblocks)), GRB.MAXIMIZE)
# definir restricciones
prec_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/marvin.prec'
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()

Optimize a model with 650631 rows, 53271 columns and 1301262 nonzeros
Variable types: 0 continuous, 53271 integer (53271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 3052245.7535
Found heuristic solution: objective 2.155104e+07
Found heuristic solution: objective 3.447631e+07
Presolve removed 583953 rows and 46469 columns (presolve time = 5s) ...
Presolve removed 617931 rows and 49095 columns
Presolve time: 6.78s
Presolved: 32700 rows, 4176 columns, 65400 nonzeros
Found heuristic solution: objective 8.477468e+08
Variable types: 0 continuous, 4176 integer (4176 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5500000e+09   0.000000e+00   4.745000e+03      7s
    5515    1.2375244e+09   0.000000e+00   0.000000e+00      7s
    5515    1.2375244e+09   0.000000e+00   0.000000e

In [173]:
# beneficio con recuperacion de Daniel y calculo posterior (economic value rec cte)
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(quicksum(economic_value_rec_list[i]*x[i] for i in range(nblocks)), GRB.MAXIMIZE)
# definir restricciones
prec_path = '/Users/administrador2/Desktop/Memoria/Codigos/minelib_inputs/marvin.prec'
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()

Optimize a model with 650631 rows, 53271 columns and 1301262 nonzeros
Variable types: 0 continuous, 53271 integer (53271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 2732201.5030
Found heuristic solution: objective 2.001547e+07
Found heuristic solution: objective 3.193380e+07
Presolve removed 548375 rows and 43446 columns (presolve time = 5s) ...
Presolve removed 587477 rows and 46778 columns (presolve time = 10s) ...
Presolve removed 615087 rows and 48834 columns
Presolve time: 12.87s
Presolved: 35544 rows, 4437 columns, 71088 nonzeros
Found heuristic solution: objective 7.832359e+08
Variable types: 0 continuous, 4437 integer (4437 binary)
Presolve removed 4 rows and 152 columns
Presolved: 4433 rows, 39829 columns, 75225 nonzeros

Presolve removed 4433 rows and 39829 columns

Root simplex log...

Iteration    Objective       Pri

In [142]:
# recuperar los bloques a sacar
indices = [i for i in range(nblocks) if x[i].x == 1]
len(indices)

In [86]:
# beneficio con valores objetivos dado por .upit
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(quicksum(bv_list[i]*x[i] for i in range(nblocks)), GRB.MAXIMIZE)
# definir restricciones
prec_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/newman1.prec'
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()
# recuperar upit
blocks_id = [i for i in range(nblocks) if x[i].x==1] # recuperar upit

Optimize a model with 3922 rows, 1060 columns and 7844 nonzeros
Variable types: 0 continuous, 1060 integer (1060 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 989307.55494
Found heuristic solution: objective 2.608690e+07
Presolve removed 3922 rows and 1060 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 16 available processors)

Solution count 2: 2.60869e+07 989308 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.608689902597e+07, best bound 2.608689902597e+07, gap 0.0000%


In [67]:
# obtener funcion objetivo del .pcpsp, para comparar
# el resultado del modelo gurobi con el modelo AMPL
pcpsp_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/'
pcpsp_name = 'newman1.pcpsp'
objective_function_pcpsp = {}
resource_constraint_ub_limits = {}
resource_constraint_lb_limits = {}
resource_constraint_coefficients = {}
with open(pcpsp_path+pcpsp_name, 'r') as f:
    linea = ''
    while not linea == 'OBJECTIVE_FUNCTION:\n':
        linea = f.readline()
        linea_lista = linea.split()
        if linea_lista[0] == 'NBLOCKS:':
            dato = linea_lista[1].strip('\n')
            nblocks = int(dato)
        elif linea_lista[0] == 'NPERIODS:':
            dato = linea_lista[1].strip('\n')
            nperiods = int(dato)
        elif linea_lista[0] == 'NDESTINATIONS:':
            dato = linea_lista[1].strip('\n')
            ndestinations = int(dato)
        elif linea_lista[0] == 'NRESOURCE_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            nresource_side_constraints = int(dato)
        elif linea_lista[0] == 'NGENERAL_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            ngeneral_side_constraints = int(dato)
        elif linea_lista[0] == 'DISCOUNT_RATE:':
            dato = linea_lista[1].strip('\n')
            discount_rate = float(dato)
        elif linea_lista[0] == 'RESOURCE_CONSTRAINT_LIMITS:':
            for r in range(nresource_side_constraints):
                for t in range(nperiods):
                    linea = f.readline()
                    linea_lista = linea.split()
                    resource_constraint_ub_limits[r,t] = int(linea_lista[-1])
                    resource_constraint_lb_limits[r,t] = '-Infinity'
    for line in f:
        if line == 'RESOURCE_CONSTRAINT_COEFFICIENTS:\n':
            break
        lista_linea = line.split()
        b = int(lista_linea[0])
        objective_function_pcpsp[b,0] = float(lista_linea[1])
        objective_function_pcpsp[b,1] = float(lista_linea[2])
    for line in f:
        if line == 'EOF\n':
            break
        linea_lista = line.split()
        b = int(linea_lista[0])
        r = int(linea_lista[1])
        d = int(linea_lista[2])
        resource_constraint_coefficients[b,r,d] = float(linea_lista[3])

In [82]:
# llenar con ceros las entradas de resource_constraint_coefficients
# que no está definidas
for b,r,d in itertools.product(range(nblocks), range(nresource_side_constraints),range(ndestinations)):
    if not (b,r,d) in resource_constraint_coefficients:
        resource_constraint_coefficients[b,r,d] = 0

In [87]:
# pcpsp model
m = Model()
# definir variables para pcpsp
x_pcpsp = {}
y = {}
for i in blocks_id:
    for t in range(nperiods):
        x_pcpsp[i,t] = m.addVar(vtype=GRB.BINARY, name = "x_pcpsp%d" % i)
for i in blocks_id:
    for d in range(ndestinations):
        for t in range(nperiods):
            y[i,d,t] = m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS)

m.setObjective(quicksum((1.0/(1.0 + discount_date))**t * objective_function_pcpsp[b,d] * y[b,d,t] for t,d,b in itertools.product(range(nperiods), range(ndestinations),blocks_id)), GRB.MAXIMIZE)
# restricciones para pcpsp
# resource contraints
for r in range(nresource_side_constraints):
    for t in range(nperiods):
        m.addConstr(resource_constraint_lb_limits[r,t] <=quicksum([resource_constraint_coefficients[b,r,d]*y[b,d,t] for b, d in itertools.product(blocks_id,range(ndestinations))])<=resource_constraint_ub_limits[r,t])
# 

In [30]:
list(itertools.product([1],range(6),range(5)))

[(1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 0, 3),
 (1, 0, 4),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (1, 1, 4),
 (1, 2, 0),
 (1, 2, 1),
 (1, 2, 2),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 0),
 (1, 3, 1),
 (1, 3, 2),
 (1, 3, 3),
 (1, 3, 4),
 (1, 4, 0),
 (1, 4, 1),
 (1, 4, 2),
 (1, 4, 3),
 (1, 4, 4),
 (1, 5, 0),
 (1, 5, 1),
 (1, 5, 2),
 (1, 5, 3),
 (1, 5, 4)]